In [1]:
! pip install -U groq

  Attempting uninstall: groq
    Found existing installation: groq 0.9.0
    Uninstalling groq-0.9.0:
      Successfully uninstalled groq-0.9.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mem0ai 0.0.5 requires groq<0.10.0,>=0.9.0, but you have groq 0.11.0 which is incompatible.


# Key Setup

In [3]:
import os
os.environ['GROQ_API_KEY'] = "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"

# client initiate

In [4]:
from groq import Groq
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# test groq model

In [5]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

In [6]:
print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be attributed to several factors:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in scenarios where timely responses are critical, such as customer support or emergency services.
2. **Low-Latency Inference**: Fast language models can perform inference (i.e., generate text or make predictions) rapidly, which is essential for applications that require immediate responses. This low-latency inference capability is critical in applications like language translation, sentiment analysis, and text summarization.
3. **Scalability**: Fast language models can handle large volumes of data and scale to meet the demands of high-traffic applications. This scalability is vital for applications like social media, online

# Setup the Agent

In [7]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [8]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE 

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

# tools

In [10]:
import re
import httpx
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


In [11]:
def calculate(operation: str) -> float:
    return eval(operation)


# Instantiation of the Agent

In [12]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

# Test agent

In [18]:
loop(query="what is c++?")

Thought: I should look up C++ on Wikipedia
Action: wikipedia: C++
PAUSE
[('wikipedia', 'C++')]
Observation: <span class="searchmatch">C</span>++ (/ˈsiː plʌs plʌs/, pronounced &quot;<span class="searchmatch">C</span> plus plus&quot; and sometimes abbreviated as CPP) is a high-level, general-purpose programming language created by Danish
Thought: It seems like I've found the answer, C++ is a high-level, general-purpose programming language.
Action: None needed, I have the answer.
Answer: C++ is a high-level, general-purpose programming language.


In [19]:
loop(query="2+2*3=?")

Thought: I need to calculate the result of 2+2*3
Action: calculate: 2+2*3
PAUSE
[('calculate', '2+2*3')]
Observation: 8
Thought: I have the answer
Action: 
Answer: 2+2*3 is 8
